> __Purpose__: Experiment with traditional ML approaches (e.g. no deep learning, keep it simple here).  Regression, binary classification (e.g. did stroke occur in this vessel y/n), and dense classification (e.g. classify every single data point into "stroke is occurring" or "stroke is not occurring")

In [1]:
import pandas as pd
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import datetime
import os

title_font_size = 30
label_font_size = 20

plt.rc('font', size=title_font_size) #controls default text size
plt.rc('axes', titlesize=title_font_size) #fontsize of the title
plt.rc('axes', labelsize=label_font_size) #fontsize of the x and y labels
plt.rc('xtick', labelsize=label_font_size) #fontsize of the x tick labels
plt.rc('ytick', labelsize=label_font_size) #fontsize of the y tick labels
plt.rc('legend', fontsize=label_font_size) #fontsize of the legend

Load data and the premade labels from the previous NB

In [2]:
# RAW DATA FILES
mat_94b = loadmat('data_94b.mat')
mat_95q = loadmat('data_95q.mat')

# LABELS
y_train_reg95 = np.load(os.path.join('Labels', '95_reg.npy'))
y_train_class95 = np.load(os.path.join('Labels', '95_class.npy'))
y_train_class_1D95 = np.load(os.path.join('Labels', '95_class_1D.npy'))

y_train_reg94b = np.load(os.path.join('Labels', '94b_reg.npy'))
y_train_class94b = np.load(os.path.join('Labels', '94b_class.npy'))
y_train_class_1D94b = np.load(os.path.join('Labels', '94b_class_1D.npy'))

In [3]:
all_mats = [mat_94b, mat_95q]
mat_names = ["94b", "95"]
num_mats = len(all_mats)

num_vessels_lst = [0] * num_mats
m_rICT = [0] * num_mats
t = [0] * num_mats

labels = [0] * num_mats

In [4]:
running_max = 0
for i, mat in enumerate(all_mats):
    num_vessels_lst[i] = mat['names'].shape[1]
    mat_name = mat_names[i]
    
    m_rICT[i] = mat['rICT']
    # Need to find what the longest rICT vector is
    if m_rICT[i].shape[0] > running_max:
        running_max = m_rICT[i].shape[0]
    #m_ROI[i] = mat['ROI']  # I don't think I actually need this, for now at least
    m_t = mat['t']
    t[i] = m_t.reshape((m_t.shape[1]))
    
    y_train_reg = np.load(os.path.join('Labels', mat_name + '_reg.npy'))
    y_train_class = np.load(os.path.join('Labels', mat_name + '_class.npy'))
    y_train_class_1D = np.load(os.path.join('Labels', mat_name + '_class_1D.npy'))
    labels[i] = [y_train_reg, y_train_class, y_train_class_1D]

In [5]:
print(y_train_reg.shape)
print(y_train_class.shape)
print(y_train_class_1D.shape)

(7,)
(7, 4000)
(7,)


Assemble the data from the different datasets into a single dataframe, and a single set of labels

In [6]:
rict_df = pd.DataFrame()
reg_labels_npy = np.array([])
class_labels_df = pd.DataFrame()
class1D_labels_npy = np.array([])

# Create the rict_df of input, and the labels_df 
for i in range(len(num_vessels_lst)):
    # First, zero pad to reach max vector length
    if running_max - m_rICT[i].shape[0] > 0:
        zp_mat = np.zeros(((running_max - m_rICT[i].shape[0]), num_vessels_lst[i]))
        zp_rict = np.concatenate((m_rICT[i], zp_mat))
        
        zp_class = np.concatenate((labels[i][1], np.transpose(zp_mat)), axis=1)
    else:
        zp_rict = m_rICT[i]
        zp_class = labels[i][1]
        
    # Now safely append to dataframe
    rict_df = pd.concat((rict_df, pd.DataFrame(np.transpose(zp_rict)))) #, axis=1
    # Labels
    reg_labels_npy = np.concatenate((reg_labels_npy, labels[i][0]))
    class_labels_df = pd.concat((rict_df, pd.DataFrame(zp_class)))
    class1D_labels_npy = np.concatenate((class1D_labels_npy, labels[i][2]))
    
    print(f"{i}: delta t is {(t[i][25] - t[i][0])/25}")

0: delta t is 0.14110399999998663
1: delta t is 0.00736


In [7]:
print(rict_df.shape)
rict_df.head()

(14, 4000)


,0,1,2,3,4,5,6,7,8,9,...,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999
0,1.061179,1.036253,1.073106,0.970642,0.957931,0.929883,0.968243,0.960815,0.965519,1.005626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.017361,0.968635,1.013525,0.959467,0.932092,0.962371,0.985138,0.994388,0.998228,1.001761,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.114077,1.059771,1.078420,0.993976,0.951927,0.979590,1.005449,1.010853,1.028106,1.053987,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.096296,1.021505,1.078318,0.988432,0.953285,0.961240,0.990814,0.992204,1.000505,1.047153,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.147843,1.105764,1.058356,1.011027,0.935990,0.957186,0.957919,0.988637,0.962370,1.026500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Make training df

In [8]:
x_train = rict_df.copy(deep=True).transpose()
x_train.head()

,0,1,2,3,4,5,6,0,1,2,3,4,5,6
0,1.061179,1.017361,1.114077,1.096296,1.147843,1.121544,1.109880,1.104496,1.065481,1.096309,1.066983,1.033309,1.050900,1.020680
1,1.036253,0.968635,1.059771,1.021505,1.105764,1.083047,1.038855,1.070832,1.069529,1.116270,1.065764,0.971102,1.073872,1.043329
2,1.073106,1.013525,1.078420,1.078318,1.058356,1.081180,1.069371,1.021953,1.009902,0.998641,1.011728,0.992180,1.027361,1.014699
3,0.970642,0.959467,0.993976,0.988432,1.011027,1.045294,0.961510,1.018994,0.991405,1.013847,1.043583,0.970549,1.035323,1.035490
4,0.957931,0.932092,0.951927,0.953285,0.935990,0.973522,0.945293,1.026853,1.005429,1.053537,1.052560,1.000330,1.069433,1.061703


## ML Modeling

In [9]:
# Machine learning
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [1]:
# Standard model fitting

def fit_ml_algo(algo, x_train, y_train, cv):
    '''Runs given algorithm and returns the accuracy metrics'''
    
    model = algo.fit(x_train, y_train)
    acc = round(model.score(x_train, y_train) * 100, 2)
    # Cross Validation 
    train_pred = model_selection.cross_val_predict(algo, 
                                                  x_train, 
                                                  y_train, 
                                                  cv=cv, 
                                                  n_jobs = -1)
    # Cross-validation accuracy metric
    acc_cv = round(metrics.accuracy_score(y_train, train_pred) * 100, 2)
    
    return train_pred, acc, acc_cv

## Regression
> https://medium.com/analytics-vidhya/5-regression-algorithms-you-need-to-know-theory-implementation-37993382122d
1. Linear Regression
2. Neural Network Regression --> Use a linear activation function on the last layer
3. Decision Tree Regression
4. LASSO Regression --> Good for data that shows heavy multicollinearity (heavy correlation of features with each other)
5. Rdige Regression --> Also good for datasets that have an abundant amount of featuesr which are not indepdent (collinearity) from one another
6. ElasticNet Regression
> https://www.jigsawacademy.com/popular-regression-algorithms-ml/
1. Random Forest
2. SVM
3. Gaussian Regression
4. Polynomial Regression
> https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics
- Metrics we care about ^^
> https://towardsdatascience.com/cyclical-features-encoding-its-about-time-ce23581845ca
- Could do this to encode the time series with the data...

In [14]:
X_train = np.transpose(x_train)
y_train_reg = reg_labels_npy

## Random Forest

In [28]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train, y_train_reg)
train_pred = model.predict(X_train)

print("ML Predictions:")
print(train_pred)
print("\nGround Truth:")
print(y_train_reg)
print(f"\nAccuracy: {round(model.score(X_train, y_train_reg) * 100, 2)}")

ML Predictions:
[  3.43   0.     0.   303.91   3.43   0.     0.     0.   266.77   0.
   0.   265.97   0.     0.  ]

Ground Truth:
[  0.   0.   0. 343.   0.   0.   0.   0. 263.   0.   0. 263.   0.   0.]

Accuracy: 99.22


Ought to look at some more informative metrics besides just accuracy, especially since this dataset is very sparse (e.g. if we predict no stroke occur we would still get high accuracy since so few stroke occur).  Need more data to do anything substantial.  __Good accuracy but main concern is overfitting, but I have no way to test this with only 2 trials of data.__

## LASSO Regression

In [29]:
from sklearn.linear_model import LassoCV
model = LassoCV()
model.fit(X_train, y_train_reg)
train_pred = model.predict(X_train)

print("ML Predictions:")
print(train_pred)
print("\nGround Truth:")
print(y_train_reg)
print(f"\nAccuracy: {round(model.score(X_train, y_train_reg) * 100, 2)}")

ML Predictions:
[ 4.06619344e+00  2.95715256e+00  2.02649554e+00  3.36799217e+02
  2.46491820e+00 -2.06867086e+00 -3.52540983e+00 -5.61155060e-02
  2.63682130e+02 -8.86686436e-01 -1.58154981e+00  2.64312430e+02
  9.02188538e-01 -9.22930689e-02]

Ground Truth:
[  0.   0.   0. 343.   0.   0.   0.   0. 263.   0.   0. 263.   0.   0.]

Accuracy: 99.95


C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.187e+02, tolerance: 2.020e+01
  model = cd_fast.enet_coordinate_descent(


In [30]:
from sklearn.linear_model import RidgeCV
model = RidgeCV()
model.fit(X_train, y_train_reg)
train_pred = model.predict(X_train)

print("ML Predictions:")
print(train_pred)
print("\nGround Truth:")
print(y_train_reg)
print(f"\nAccuracy: {round(model.score(X_train, y_train_reg) * 100, 2)}")

ML Predictions:
[ 32.66270264  -2.58082659   4.32832252 248.2312037   57.47264266
 -12.66758408  -7.75493658 -22.45318082 291.31659592  19.02932844
 -18.68029234 268.63907098   2.02238986   9.4345637 ]

Ground Truth:
[  0.   0.   0. 343.   0.   0.   0.   0. 263.   0.   0. 263.   0.   0.]

Accuracy: 92.21


## ElasticNet Regression
>Does not converge, do not run

In [ ]:
#from sklearn.linear_model import ElasticNetCV
#model = ElasticNetCV()
#model.fit(X_train, y_train_reg)
#train_pred = model.predict(X_train)

#print("ML Predictions:")
#print(train_pred)
#print("\nGround Truth:")
#print(y_train_reg)
#print(f"\nAccuracy: {round(model.score(X_train, y_train_reg) * 100, 2)}")

# Classification
> The way it is currently set up, it only tells you if the vessel stroked, but not when... if you want it to classify each point you have to change the input data shape, but I think that would remove the temporal relationships, assuming the model is using that.

In [52]:
#reg_labels_npy = np.array([])
#class_labels_df = pd.DataFrame()
#class1D_labels_npy = np.array([])

## Logistc Regression (this is classification)

In [36]:
# Logistic Regression
start_time = time.time()

#####################################################################

#fit_ml_algo(algo, x_train, y_train, cv):   
algo = LogisticRegression()
X_train = np.transpose(x_train)
y_train_class = np.transpose(class1D_labels_npy)
cv = 10

model = algo.fit(X_train, y_train_class)
acc = round(model.score(X_train, y_train_class) * 100, 2)
# Cross Validation 
train_pred = model_selection.cross_val_predict(algo, 
                                              X_train, 
                                              y_train_class, 
                                              cv=cv, 
                                              n_jobs = -1)
# Cross-validation accuracy metric
acc_cv = round(metrics.accuracy_score(y_train_class, train_pred) * 100, 2)

#return train_pred, acc, acc_cv

#####################################################################

log_time = (time.time() - start_time)
print("Accuracy: %s" % acc)
print("Accuracy CV 10-Fold: %s" % acc_cv)
print("Run Time: %s" % datetime.timedelta(seconds=log_time))

C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 100.0
Accuracy CV 10-Fold: 100.0
Run Time: 0:00:00.469448


In [37]:
metrics.confusion_matrix(y_train_class, train_pred)

array([[11,  0],
       [ 0,  3]], dtype=int64)

In [38]:
train_pred

array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.])

Also logistic regression but using my function for comparison

In [39]:
# Logistic Regression
start_time = time.time()
train_pred_log, acc_log, acc_cv_log = fit_ml_algo(LogisticRegression(), X_train, y_train_class, 10)
log_time = (time.time() - start_time)
print("Accuracy: %s" % acc_log)
print("Accuracy CV 10-Fold: %s" % acc_cv_log)
print("Run Time: %s" % datetime.timedelta(seconds=log_time))

C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 100.0
Accuracy CV 10-Fold: 100.0
Run Time: 0:00:00.574094


## K-Nearest Neighbours


In [40]:
# k-Nearest Neighbours
start_time = time.time()
train_pred_knn, acc_knn, acc_cv_knn = fit_ml_algo(KNeighborsClassifier(), 
                                                  X_train, 
                                                  y_train_class, 
                                                  10)
knn_time = (time.time() - start_time)
print("Accuracy: %s" % acc_knn)
print("Accuracy CV 10-Fold: %s" % acc_cv_knn)
print("Running Time: %s" % datetime.timedelta(seconds=knn_time))

C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 100.0
Accuracy CV 10-Fold: 78.57
Running Time: 0:00:00.778337


## Gaussian Naive Bayes

In [41]:
# Gaussian Naive Bayes
start_time = time.time()
train_pred_gaussian, acc_gaussian, acc_cv_gaussian = fit_ml_algo(GaussianNB(), 
                                                                      X_train, 
                                                                      y_train_class, 
                                                                           10)
gaussian_time = (time.time() - start_time)
print("Accuracy: %s" % acc_gaussian)
print("Accuracy CV 10-Fold: %s" % acc_cv_gaussian)
print("Running Time: %s" % datetime.timedelta(seconds=gaussian_time))

C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 100.0
Accuracy CV 10-Fold: 78.57
Running Time: 0:00:00.371423


## Linear SVC

In [43]:
# Linear SVC
start_time = time.time()
train_pred_svc, acc_linear_svc, acc_cv_linear_svc = fit_ml_algo(LinearSVC(),
                                                                X_train, 
                                                                y_train_class, 
                                                                10)
linear_svc_time = (time.time() - start_time)
print("Accuracy: %s" % acc_linear_svc)
print("Accuracy CV 10-Fold: %s" % acc_cv_linear_svc)
print("Running Time: %s" % datetime.timedelta(seconds=linear_svc_time))

C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 100.0
Accuracy CV 10-Fold: 100.0
Running Time: 0:00:00.576586


## Stochastic Gradient Descent


In [44]:
# Stochastic Gradient Descent
start_time = time.time()
train_pred_sgd, acc_sgd, acc_cv_sgd = fit_ml_algo(SGDClassifier(), 
                                                  X_train, 
                                                  y_train_class,
                                                  10)
sgd_time = (time.time() - start_time)
print("Accuracy: %s" % acc_sgd)
print("Accuracy CV 10-Fold: %s" % acc_cv_sgd)
print("Running Time: %s" % datetime.timedelta(seconds=sgd_time))

C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 100.0
Accuracy CV 10-Fold: 92.86
Running Time: 0:00:00.416241


## Decision Tree Classifier


In [45]:
# Decision Tree Classifier
start_time = time.time()
train_pred_dt, acc_dt, acc_cv_dt = fit_ml_algo(DecisionTreeClassifier(), 
                                                                X_train, 
                                                                y_train_class,
                                                                10)
dt_time = (time.time() - start_time)
print("Accuracy: %s" % acc_dt)
print("Accuracy CV 10-Fold: %s" % acc_cv_dt)
print("Running Time: %s" % datetime.timedelta(seconds=dt_time))

C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 100.0
Accuracy CV 10-Fold: 85.71
Running Time: 0:00:00.399156


## Gradient Boosting Trees

In [46]:
# Gradient Boosting Trees
start_time = time.time()
train_pred_gbt, acc_gbt, acc_cv_gbt = fit_ml_algo(GradientBoostingClassifier(), 
                                                                       X_train, 
                                                                       y_train_class,
                                                                       10)
gbt_time = (time.time() - start_time)
print("Accuracy: %s" % acc_gbt)
print("Accuracy CV 10-Fold: %s" % acc_cv_gbt)
print("Running Time: %s" % datetime.timedelta(seconds=gbt_time))

C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(


Accuracy: 100.0
Accuracy CV 10-Fold: 100.0
Running Time: 0:00:01.354172
